In [3]:
#Import libraries
os.chdir("/home/robbe/ionbot/")
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import re
import numpy as np
import os
from pathlib import Path
from pyteomics import mass as pymass
import OxiAnalysis as OA
import importlib
importlib.reload(OA)
db = pymass.Unimod()
modslist = []
for p in range(len(db.mods)):
    for pp in db.mods[p]['specificity']:
        if db.mods[p]['record_id'] in [35, 53, 129, 130, 205, 206, 275, 288, 318, 319, 335, 340, 342, 344, 345, 348, 349, 350, 351, 352, 354, 
        359, 360, 368, 378, 392, 401, 421, 425, 534, 540, 548, 569, 720, 721, 743, 743, 860, 936, 936, 937, 949, 1384, 1914, 1915, 1916, 1917, 1918, 
        1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929]:
            t = db.mods[p]['title']
            t = t.replace("[",":").replace("]",":")
            mod = "[" + str(db.mods[p]['record_id']) + "]" + t + "[" + pp['site'] + "]"
            modslist.append(mod)
modslist.append('[35]oxidation[M]')


In [4]:
for i in modslist:
    print(i)

[35]Oxidation[T]
[35]Oxidation[E]
[35]Oxidation[S]
[35]Oxidation[Q]
[35]Oxidation[L]
[35]Oxidation[I]
[35]Oxidation[U]
[35]Oxidation[G]
[35]Oxidation[W]
[35]Oxidation[C]
[35]Oxidation[H]
[35]Oxidation[V]
[35]Oxidation[R]
[35]Oxidation[M]
[35]Oxidation[Y]
[35]Oxidation[F]
[35]Oxidation[P]
[35]Oxidation[N]
[35]Oxidation[K]
[35]Oxidation[D]
[53]HNE[K]
[53]HNE[H]
[53]HNE[C]
[53]HNE[A]
[53]HNE[L]
[359]Pro->pyro-Glu[P]
[348]His->Asn[H]
[349]His->Asp[H]
[350]Trp->Hydroxykynurenin[W]
[368]Cys->Dha[C]
[344]Arg->GluSA[R]
[345]Trioxidation[Y]
[345]Trioxidation[W]
[345]Trioxidation[C]
[345]Trioxidation[F]
[354]Nitro[Y]
[354]Nitro[W]
[354]Nitro[F]
[351]Trp->Kynurenin[W]
[352]Lys->Allysine[K]
[129]Iodo[H]
[129]Iodo[Y]
[130]Diiodo[Y]
[130]Diiodo[H]
[360]Pro->Pyrrolidinone[P]
[205]Delta:H(6)C(6)O(1)[K]
[206]Delta:H(4)C(3)O(1)[K]
[206]Delta:H(4)C(3)O(1)[H]
[206]Delta:H(4)C(3)O(1)[C]
[206]Delta:H(4)C(3)O(1)[R]
[340]Bromo[F]
[340]Bromo[H]
[340]Bromo[W]
[340]Bromo[Y]
[342]Amino[Y]
[275]Nitrosyl[C]
[275]Ni

In [5]:
def modratios(df):
    """
    Makes modratios dataframe of occurrence of oxPTM/total PSMs
    Parameter: df
    """
    import pandas as pd
    matched_peptide = df["matched_peptide"]
    AAdict = {"A" : 0, "R" : 0, "N" : 0, "D" : 0, "C" : 0, "Q" : 0, "E" : 0, "G" : 0, "H" : 0, "I" : 0, "K" : 0, "M" : 0, "F" : 0, "P" : 0, "S" : 0, "T" : 0, "W" : 0, "Y" : 0, "V" : 0}
    for i, peptide in matched_peptide.items():
        for AA in peptide:
            AAdict[AA] += 1  
    return AAdict      
    modifications = df["Modification"]
    Modification = []
    Ratios = []
    for mod in modslist:
        pattern = re.compile(r"\[\D+\]")
        modified_AA = re.findall(pattern, mod)
        modified_AA = modified_AA[0]
        modified_AA = list(modified_AA)[1]
    
        Modification.append(mod)
        val = 0
        total = AAdict[modified_AA] if modified_AA in AAdict.keys() else 1
        for i, modification in modifications.items():
            if modification == None:
                continue
            elif mod in modification:
                val += 1
        Ratios.append(val/total)
    dt = {"Modification" : Modification, "Ratios" : Ratios}
    result = pd.DataFrame(dt)
    return result

In [6]:
first = pd.read_csv("/home/robbe/ionbot/full_projects/PXD022545_first.csv", index_col=0)
del first["Unnamed: 0"]
del first["Unnamed: 0.1"]
del first["spectrum_title"]
del first["scan"]
first['modifications'] = first['modifications'].fillna("None")
first['Oxidant treatment'] = first['Oxidant treatment'].fillna("None")
first['Secondary treatment'] = first['Secondary treatment'].fillna("None")
first['Length of Secondary treatment (mins)'] = first['Length of Secondary treatment (mins)'].fillna(0)
first['Length of oxidant treatment (mins)'] = first['Length of oxidant treatment (mins)'].replace('0', 0)

/tmp/ipykernel_2917840/4217098548.py:1: DtypeWarning: Columns (25,29) have mixed types. Specify dtype option on import or set low_memory=False.
  first = pd.read_csv("/home/robbe/ionbot/full_projects/PXD022545_first.csv", index_col=0)


In [7]:
first['Peptidoform_name'] = first.apply(lambda row: OA.peptidoform_name(row), axis=1)
first['Position'] = first['modifications'].apply(OA.get_positions)
first['Modification'] = first['modifications'].apply(OA.get_modification)
first['Oxidatively_modified'] = first['modifications'].apply(OA.oxidatively_modified)

In [8]:
H2O2 = first.loc[lambda first: first["Oxidant treatment"] == "H2O2"]
Control = first.loc[lambda first: first["Oxidant treatment"] == "None"]
print("There are this many replicates of each treatment: {}".format(H2O2["spectrum_file"].nunique()))
print("The H2O2 treatment contains this many PSMs: {}".format(H2O2.shape[0]))
print("The Control treatment contains this many PSMs: {}".format(Control.shape[0]))

There are this many replicates of each treatment: 3
The H2O2 treatment contains this many PSMs: 38086
The Control treatment contains this many PSMs: 72283


In [9]:
H2O2rep = OA.replicate_filter(H2O2, 3)
Controlrep = OA.replicate_filter(Control, 3)

In [10]:
"""
Ik wil kijken of er geoxideerde PSMs zijn die niet geoxideerd voorkomen in de andere set
Mss moet ik ook kijken of ze ongeoxideerd voorkomen in de originele set, al denk ik dat als de ene voorkomt de andere waarschijnlijk ook sws voorkomt
Op dit moment werkt het zoals het hoort, maar niet-oxidatieve mods spelen parten bv. 'APNTPASGANGD[952]Cation:Fe:II:GSM[35]oxidationSQTQSGSTVK' komt ook in
lijst terecht terwijl de oxidatieve modificatie op plaats 15 zeker ook terugkomt in control, alleen niet in combinatie met mod 952
Voorlopig heb ik dit opgelost door slechts 1 modificatie toe te laten
"""
#Oxidatively modified PSMs in H2O2
Oxmod = H2O2rep[H2O2rep["Oxidatively_modified"] == True]
#Set of oxidatively modified peptidoforms
Oxmoddedset = set(Oxmod["Peptidoform_name"])
#Set of the base matched_peptides of these oxidatively modified peptidoforms
nonoxmoddedset = set(Oxmod["matched_peptide"])
#Oxidatively modified PSMs in control
Oxmodcontrol = Controlrep[Controlrep["Oxidatively_modified"] == True]
#Set of oxidatively modified peptidoforms
Oxmoddedcontrolset = set(Oxmodcontrol["Peptidoform_name"])
#Set of the base matched_peptides of these all peptidoforms
nonoxmoddedcontrolset = set(Controlrep["matched_peptide"])
#Oxidatively modified PSMs that occur in treatment but not in control
notoxmoddedincontrol = Oxmoddedset.difference(Oxmoddedcontrolset)


list = []
for i in notoxmoddedincontrol:     
    matched_peptide = H2O2[H2O2["Peptidoform_name"] == i].iloc[0]["matched_peptide"]
    pattern = re.compile(r"\[\d+\]")
    listofmods = re.findall(pattern, i)
    n_of_mods = len(listofmods)
    if n_of_mods > 1:
        continue
    else:
        if matched_peptide in nonoxmoddedcontrolset:
            list.append(i)

print(list)

# pattern = re.compile(r"\[\d+\]")
# listofmods = re.findall(pattern, i)
# for mod in listofmods:
#     if mod in ['[35]', '[53]', '[129]', '[130]', '[205]', '[206]', '[275]', '[288]', '[318]', '[319]', '[335]', '[340]', '[342]', '[344]', '[345]', '[348]', '[349]', '[350]', '[351]', '[352]', '[354]', '[359]', '[360]', '[368]', '[378]', '[392]', '[401]', '[421]', '[425]', '[534]', '[540]', '[548]', '[569]', '[720]', '[721]', '[743]', '[860]', '[936]', '[937]', '[949]', '[1384]', '[1914]', '[1915]', '[1916]', '[1917]', '[1918]', '[1922]', '[1923]', '[1924]', '[1925]', '[1926]', '[1927]', '[1928]', '[1929]']:
#         continue
#     else:
#         break

['WAEIIPIIQQC[345]TrioxidationQVVR', 'VFTSW[425]DioxidationTGGGAAASR', 'I[35]OxidationMNVIGEPIDER', 'YM[35]oxidationPPEEGPYK', 'IQEM[425]DioxidationEGTVK', 'DQ[1918]CarbonylNYPGAIAIHHPNVAEK', 'RTEM[425]DioxidationENEFVIIK', 'AM[35]oxidationEAVAAQGK', 'DDPHAC[345]TrioxidationYSTVFDK', 'DIYANTVISGGT[35]OxidationTMYPGIADR', 'VFTSW[351]Trp->KynureninTGGGAAASR', 'GVSGSCNI[1918]CarbonylDVVCPEGDGR', 'FSHEEIAM[425]DioxidationATVTAIR', 'IETIEVM[35]oxidationEDR', 'DQNYPGAIAIHHP[425]DioxidationNVAEK', 'ADIEM[35]oxidationPFDPSK', 'DQNYPGAIAIHHP[359]Pro->pyro-GluNVAEK', 'TVWDWEIM[35]oxidationNDIK', 'DNIAEDIM[425]DioxidationR', 'EITFENGEEITEEGIPFIIIFHM[35]oxidationK', 'NAFAC[345]TrioxidationFDEEATGTIQEDYIR', 'ASVIFDTM[35]oxidationQHHIAINR', 'IM[35]oxidationIIIEVISGER', 'FDGIIGM[35]oxidationAYPR', 'SGTIAC[368]Cys->DhaTGSIVNNTANDRK', 'IKPDPNTIC[345]TrioxidationDEFK', 'EAFNM[425]DioxidationIDQNR', 'EITAIAPSTMK[35]Oxidation', 'SNNVEM[35]oxidationDWVIK']


In [11]:
Controlrep[Controlrep["matched_peptide"] == "IKPDPNTICDEFK"]

,ionbot_match_id,spectrum_file,precursor_mass,peptide_mass,observed_retention_time,charge,database_peptide,matched_peptide,modifications,modifications_delta,...,Secondary treatment,Secondary treatment dose (µM),Length of oxidant treatment (mins),Length of Secondary treatment (mins),Notes,First/Lower/Coeluting,Peptidoform_name,Position,Modification,Oxidatively_modified
9514,0_29638_1,2020-07-14_MS_PG_Proteome_Davide_Casserini_CTR...,1576.74789,15187.3881,3618.14688,3,IKPDPNTICDEFK,IKPDPNTICDEFK,9|[6]Carboxymethyl[C],9|58.0055,...,None,0.0,0,0.0,NaN,First,IKPDPNTIC[6]CarboxymethylDEFK,9,[6]Carboxymethyl[C],False
9655,0_29962_1,2020-07-14_MS_PG_Proteome_Davide_Casserini_CTR...,1576.74770,15187.3881,3648.84270,3,IKPDPNTICDEFK,IKPDPNTICDEFK,9|[6]Carboxymethyl[C],9|58.0055,...,None,0.0,0,0.0,NaN,First,IKPDPNTIC[6]CarboxymethylDEFK,9,[6]Carboxymethyl[C],False
9658,0_29968_1,2020-07-14_MS_PG_Proteome_Davide_Casserini_CTR...,1576.74900,15187.3881,3649.43994,2,IKPDPNTICDEFK,IKPDPNTICDEFK,9|[6]Carboxymethyl[C],9|58.0055,...,None,0.0,0,0.0,NaN,First,IKPDPNTIC[6]CarboxymethylDEFK,9,[6]Carboxymethyl[C],False
9942,0_30499_1,2020-07-14_MS_PG_Proteome_Davide_Casserini_CTR...,1598.69871,15187.3881,3704.97084,2,IKPDPNTICDEFK,IKPDPNTICDEFK,4|[21]Phospho[D],4|79.9663,...,None,0.0,0,0.0,NaN,First,IKPD[21]PhosphoPNTICDEFK,4,[21]Phospho[D],False
31873,0_28750_1,2020-07-14_MS_PG_Proteome_Davide_Casserini_CTR...,1576.74752,15187.3881,3616.27494,3,IKPDPNTICDEFK,IKPDPNTICDEFK,9|[6]Carboxymethyl[C],9|58.0055,...,None,0.0,0,0.0,NaN,First,IKPDPNTIC[6]CarboxymethylDEFK,9,[6]Carboxymethyl[C],False
32000,0_29048_1,2020-07-14_MS_PG_Proteome_Davide_Casserini_CTR...,1576.74624,15187.3881,3646.98534,3,IKPDPNTICDEFK,IKPDPNTICDEFK,9|[6]Carboxymethyl[C],9|58.0055,...,None,0.0,0,0.0,NaN,First,IKPDPNTIC[6]CarboxymethylDEFK,9,[6]Carboxymethyl[C],False
32249,0_29509_1,2020-07-14_MS_PG_Proteome_Davide_Casserini_CTR...,1598.69956,15187.3881,3696.28728,2,IKPDPNTICDEFK,IKPDPNTICDEFK,4|[21]Phospho[D],4|79.9663,...,None,0.0,0,0.0,NaN,First,IKPD[21]PhosphoPNTICDEFK,4,[21]Phospho[D],False
56178,0_30374_1,2020-07-14_MS_PG_Proteome_Davide_Casserini_CTR...,1576.74587,15187.3881,3625.52784,3,IKPDPNTICDEFK,IKPDPNTICDEFK,9|[6]Carboxymethyl[C],9|58.0055,...,None,0.0,0,0.0,NaN,First,IKPDPNTIC[6]CarboxymethylDEFK,9,[6]Carboxymethyl[C],False
56507,0_30934_1,2020-07-14_MS_PG_Proteome_Davide_Casserini_CTR...,1598.70055,15187.3881,3682.91706,2,IKPDPNTICDEFK,IKPDPNTICDEFK,4|[21]Phospho[D],4|79.9663,...,None,0.0,0,0.0,NaN,First,IKPD[21]PhosphoPNTICDEFK,4,[21]Phospho[D],False


In [12]:
def differentially_oxidized_psms(treatmentdf, controldf):
    
    #Oxidatively modified PSMs in H2O2
    Oxmod = treatmentdf[treatmentdf["Oxidatively_modified"] == True]
    #Set of oxidatively modified peptidoforms
    Oxmoddedset = set(Oxmod["Peptidoform_name"])
    #Set of the base matched_peptides of these oxidatively modified peptidoforms
    nonoxmoddedset = set(Oxmod["matched_peptide"])
    #Oxidatively modified PSMs in control
    Oxmodcontrol = controldf[controldf["Oxidatively_modified"] == True]
    #Set of oxidatively modified peptidoforms
    Oxmoddedcontrolset = set(Oxmodcontrol["Peptidoform_name"])
    #Set of the base matched_peptides of these all peptidoforms
    nonoxmoddedcontrolset = set(controldf["matched_peptide"])
    #Oxidatively modified PSMs that occur in treatment but not in control
    notoxmoddedincontrol = Oxmoddedset.difference(Oxmoddedcontrolset)

    list = []
    for i in notoxmoddedincontrol:     
        matched_peptide = treatmentdf[treatmentdf["Peptidoform_name"] == i].iloc[0]["matched_peptide"]
        pattern = re.compile(r"\[\d+\]")
        listofmods = re.findall(pattern, i)
        n_of_mods = len(listofmods)
        if n_of_mods > 1:
            continue
        else:
            if matched_peptide in nonoxmoddedcontrolset:
                list.append(i)
    result = "There are {} PSMs that are oxidized in the treatment data that are not oxidized in the control data".format(len(list))
    return list, result
        

In [13]:
differentially_oxidized_psms(H2O2rep, Controlrep)

(['WAEIIPIIQQC[345]TrioxidationQVVR',
  'VFTSW[425]DioxidationTGGGAAASR',
  'I[35]OxidationMNVIGEPIDER',
  'YM[35]oxidationPPEEGPYK',
  'IQEM[425]DioxidationEGTVK',
  'DQ[1918]CarbonylNYPGAIAIHHPNVAEK',
  'RTEM[425]DioxidationENEFVIIK',
  'AM[35]oxidationEAVAAQGK',
  'DDPHAC[345]TrioxidationYSTVFDK',
  'DIYANTVISGGT[35]OxidationTMYPGIADR',
  'VFTSW[351]Trp->KynureninTGGGAAASR',
  'GVSGSCNI[1918]CarbonylDVVCPEGDGR',
  'FSHEEIAM[425]DioxidationATVTAIR',
  'IETIEVM[35]oxidationEDR',
  'DQNYPGAIAIHHP[425]DioxidationNVAEK',
  'ADIEM[35]oxidationPFDPSK',
  'DQNYPGAIAIHHP[359]Pro->pyro-GluNVAEK',
  'TVWDWEIM[35]oxidationNDIK',
  'DNIAEDIM[425]DioxidationR',
  'EITFENGEEITEEGIPFIIIFHM[35]oxidationK',
  'NAFAC[345]TrioxidationFDEEATGTIQEDYIR',
  'ASVIFDTM[35]oxidationQHHIAINR',
  'IM[35]oxidationIIIEVISGER',
  'FDGIIGM[35]oxidationAYPR',
  'SGTIAC[368]Cys->DhaTGSIVNNTANDRK',
  'IKPDPNTIC[345]TrioxidationDEFK',
  'EAFNM[425]DioxidationIDQNR',
  'EITAIAPSTMK[35]Oxidation',
  'SNNVEM[35]oxidationD